In [42]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt
import copy
import scipy
from errorResult import MSE
from errorResult import RMSE
from errorResult import jdwucha
from errorResult import xdwucha
from errorResult import fanguiy
from errorResult import result_alt
from errorResult import drawAll
from errorResult import drawAll_alt

def f(lmd,c):
    # return scipy.special.expit(-lmd*c)
    y = 1 / (1 + np.exp(-0.5*c))
    return y

def h(err):
    y = 1 / (2 * err + 1)
    return y

def errorLp(p,data_pre,data_real):
    dist = np.linalg.norm(data_pre-data_real,ord=p)/(data_pre.shape[0]*data_pre.shape[1])
    return dist
  
#%%
#返回初始化权重参数   NxN 
def initialize_parameters_he(npop,N):
    parameters = np.zeros((npop,N*N))
    for i in range(npop):
        parameter = np.random.uniform(-1,1,N*(N-1))
        #np.random.randn(N*(N-1)) * np.sqrt(2.0 / (N*(N-1)))
        lmd = np.random.uniform(0,1,N)
        #lmd = np.array([1]*N)
        parameters[i] = parameter.tolist()+lmd.tolist()
    return parameters
    
def rshape(X,N):
    temp = copy.copy(X)
    e = np.zeros((N,N))
    param = temp[:-N]   
    lmd = temp[-N:]
    num = 0
    for i in range(N):
        for j in range(N):
            if i is not j:
                e[i,j] = param[num]
                num = num + 1
    return e,lmd
    
#jaya学习算法训练FCM参数
def jayaTrain(c_data,c_real,time,N,npop=4):
    X = initialize_parameters_he(npop,N)
    print(rshape(X[1,:],N))
#    for n in range(npop):
#        for i in range(N*N):
#            X[n,i] = min(X[:,i]) + np.random.random()*(max(X[:,i])-min(X[:,i]))
    fitness = np.zeros(npop)
    error = np.zeros(npop)
    worst= 0
    best = 0
    error_time = []
    c_pre = np.zeros((c_real.shape[0],c_real.shape[1]))
    for num in range(500):
        for n in range(npop):
            e,lmd = rshape(X[n,:],N)
            #print("c_data.shape: %s" % c_data.shape)
            c_pre = fcm(e,lmd,c_data,c_real,time)
            #print("c_pre, c_real: %s ,%s" % c_pre.shape, c_real.shape)
            fitness[n] = h(errorLp(2,c_pre,c_real)) 
            error[n] = errorLp(2,c_pre,c_real)
        worst = fitness.tolist().index(min(fitness))
        best = fitness.tolist().index(max(fitness))
        print(worst,best,fitness)
    
        Xx = np.zeros((npop,N*N))
        for n in range(npop):
            rand1 = np.random.uniform(0,1)
            rand2 = np.random.uniform(0,1)
            Xx[n,:] = X[n,:] + (X[best,:]-np.abs(X[n,:]))*rand1-(X[worst,:]-np.abs(X[n,:]))*rand2
            '''
            for i in range(N*N):
                if Xx[n,i] < min(Xx[:,i]):
                    Xx[n,i] = min(Xx[:,i])
                if Xx[n,i] > max(Xx[:,i]):
                    Xx[n,i] = max(Xx[:,i])
            '''
            e,lmd = rshape(Xx[n,:],N)
            c_pre = fcm(e,lmd,c_data,c_real,time)
            error2 = errorLp(2,c_pre,c_real)
            if error2 <= error[n]:
                X[n,:] = Xx[n,:]              
        error_time.append(error[best])
    e,lmd = rshape(X[best,:],N)
    return e,lmd,error_time

#模糊认知图递推过程      
def fcm(e,lmd,data_front,data,time):
    '''
    c_len = len(data)
    data_pre = np.zeros((time+1,c_len))
    data_pre[0] = data_front
    
    for t in range(0,time):
        for i in range(c_len):
            for j in range(c_len):
                if i is not j :
                    data_pre[t+1,i] += e[j,i]*data[t,j]
            data_pre[t+1,i] += data[t,i]
            data_pre[t+1,i] = f(lmd[i],data[t+1,i])
    
    '''
    c_len = len(data_front)
    data_pre = np.zeros((time,c_len))
    data_pre[0] = np.dot(data_front,e)
    for i in range(c_len):
        data_pre[0,i] = f(lmd[i],data_pre[0,i])
    for t in range(0,time-1):
        data_pre[t+1] = np.dot(data[t],e)
        for i in range(c_len):
            data_pre[t+1,i] = f(lmd[i],data_pre[t+1,i])
    return data_pre

#模糊认知图递推过程训练用   
def fcmm(e,lmd,data_front):
    c_len = len(data_front)
    data_pre = np.dot(data_front,e)
    for i in range(c_len):
        data_pre[i] = f(lmd[i],data_pre[i])
    
    return data_pre

ImportError: cannot import name 'result_alt' from 'errorResult' (E:\gitfile\FCM\errorResult.py)

In [43]:
#%% 
if __name__ == "__main__":
    #data = pd.read_csv('dataProcess.csv',index_col = 0)
    data = pd.read_csv('数据20171205/alternative1.csv',index_col = 0)
    data = np.array(data)
    data = (data-data.min())/(data.max()-data.min())
    train_time = 25               #设置滑动窗口
    start = 1
    hour = 10 
    concept = data.shape[1]
    
    #data_train = data[240:600,:]   #20天做训练
    data_test = data[start+train_time:start+train_time+hour,:]     #3天做测试
    data_pre = np.zeros((data_test.shape[0],data_test.shape[1]))
    data_train = data[start:train_time+start,:]
    
    e,lmd,error_time = jayaTrain(data[start-1,:],data_train,train_time,concept,npop = 20)
    
    for i in range(0,hour):
        #data_train = data[i+start:i+train_time+start,:]
        data_real = data[i+start+train_time-1,:]
        #e,lmd,error_time = jayaTrain(data[i+start-1,:],data_train,train_time,concept,npop = 8)
        data_pre[i] = fcmm(e,lmd,data_real)

(array([[ 0.        , -0.69866147, -0.41344833,  0.21908696, -0.07686853],
       [ 0.60811504,  0.        , -0.87588838,  0.90327481, -0.64658997],
       [ 0.68466246, -0.80058533,  0.        ,  0.84429835, -0.01689759],
       [ 0.82384805,  0.51131811,  0.56872889,  0.        ,  0.49482715],
       [-0.39693685, -0.25942797, -0.24418373,  0.80949796,  0.        ]]), array([0.61590093, 0.9467507 , 0.41756614, 0.18374535, 0.6664757 ]))
11 16 [0.97579958 0.9692091  0.97706083 0.97275015 0.97613752 0.97587422
 0.97358233 0.97188708 0.97652243 0.97600765 0.97536909 0.96760899
 0.97309194 0.97258764 0.97111226 0.97467524 0.97895882 0.97255832
 0.97352842 0.97319588]
14 5 [0.97579958 0.97537722 0.98085116 0.97940554 0.98048286 0.98092405
 0.97598211 0.97920464 0.97927692 0.97870173 0.98046599 0.9787334
 0.97309194 0.97258764 0.97111226 0.97821923 0.97936213 0.97255832
 0.97666668 0.97319588]
17 5 [0.97579958 0.97957389 0.98085116 0.97940554 0.98065788 0.98092405
 0.97598211 0.97920464 0.9

2 11 [0.98183508 0.98207072 0.9811972  0.98198608 0.9820488  0.98231565
 0.98249865 0.98175811 0.98251782 0.98164819 0.98137276 0.98257584
 0.98187562 0.98214033 0.98229117 0.98228209 0.98219811 0.98121844
 0.98227564 0.98223184]
2 11 [0.98183508 0.98207072 0.9811972  0.98198608 0.9820488  0.98231565
 0.98249865 0.98175811 0.98251782 0.98164819 0.98137276 0.98257584
 0.98187562 0.98214033 0.98229117 0.98228209 0.98219811 0.98121844
 0.98227564 0.98223184]
17 11 [0.98183508 0.98207072 0.98174345 0.98198608 0.9820488  0.98231565
 0.98249865 0.98200373 0.98251782 0.98164819 0.98137276 0.98257584
 0.98187562 0.98214033 0.98229117 0.98228209 0.98219811 0.98121844
 0.98234708 0.98223184]
17 11 [0.98183508 0.98207072 0.98174345 0.98198608 0.9820488  0.98231565
 0.98249865 0.98200373 0.98251782 0.98172358 0.98137276 0.98257584
 0.98187562 0.98214033 0.98229117 0.98228209 0.98219811 0.98121844
 0.98234708 0.98223184]
17 11 [0.98183508 0.98207072 0.98174345 0.98198608 0.9820488  0.98231565
 0.98

15 17 [0.98322635 0.9829717  0.98308946 0.98340915 0.98285143 0.98276619
 0.98249865 0.98264166 0.98298791 0.98253186 0.98276301 0.98309683
 0.98266187 0.98265789 0.98229117 0.98228209 0.98289662 0.98341884
 0.98283339 0.98291135]
15 17 [0.98322635 0.9829717  0.98308946 0.98340915 0.98285143 0.98276619
 0.98249865 0.98264166 0.98298791 0.98253186 0.98276301 0.98309683
 0.98266187 0.98268634 0.98229117 0.98228209 0.98289662 0.98341884
 0.98283339 0.98291135]
15 17 [0.98322635 0.9829717  0.98308946 0.98340915 0.98285143 0.98276619
 0.98249865 0.98332116 0.98298791 0.98253186 0.98276301 0.98309683
 0.98266187 0.98268634 0.98229117 0.98228209 0.98289662 0.98341884
 0.98283339 0.98291135]
15 17 [0.98322635 0.9829717  0.98308946 0.98340915 0.98285143 0.98276619
 0.98249865 0.98332116 0.98298791 0.98253186 0.98276301 0.98309683
 0.98266187 0.98268634 0.98229117 0.98228209 0.98289662 0.98341884
 0.98283339 0.98291135]
15 17 [0.98322635 0.9829717  0.98308946 0.98340915 0.98285143 0.98276619
 0.

13 15 [0.98322635 0.98300963 0.98308946 0.98348652 0.98329223 0.98298174
 0.98377324 0.98332116 0.98348628 0.98285503 0.98316752 0.98357706
 0.98310043 0.98268634 0.9834047  0.98385422 0.98289662 0.98341884
 0.98283339 0.98350253]
13 8 [0.98322635 0.98300963 0.98308946 0.98348652 0.98329223 0.98298174
 0.98377324 0.98332116 0.98389752 0.98285503 0.98316752 0.98357706
 0.98310043 0.98268634 0.9834047  0.98385422 0.98289662 0.98341884
 0.98283339 0.98350253]
18 8 [0.98322635 0.98300963 0.98308946 0.98348652 0.98329223 0.98298174
 0.98377324 0.98332116 0.98389752 0.98285503 0.98316752 0.98357706
 0.98313672 0.98333735 0.9834047  0.98385422 0.98289662 0.98341884
 0.98283339 0.98350253]
18 8 [0.98322635 0.98300963 0.98308946 0.98348652 0.98329223 0.98298174
 0.98377324 0.98332116 0.98389752 0.98285503 0.98316752 0.98357706
 0.98313672 0.98333735 0.9834047  0.98385422 0.98289662 0.98341884
 0.98283339 0.98350253]
18 8 [0.98322635 0.98300963 0.98308946 0.98348652 0.98329223 0.98298174
 0.9837

9 4 [0.98322635 0.98376154 0.98316306 0.98348652 0.98403691 0.98343998
 0.98377324 0.98332116 0.98389752 0.98293333 0.98353404 0.98357706
 0.98313672 0.98337687 0.9834047  0.98385422 0.98349379 0.98345116
 0.98391697 0.98350253]
9 4 [0.98322635 0.98376154 0.98316306 0.98348652 0.98403691 0.98343998
 0.98377324 0.98332116 0.98389752 0.98293333 0.98353404 0.98357706
 0.98313672 0.98337687 0.9834047  0.98385422 0.98349379 0.98345116
 0.98391697 0.98350253]
9 4 [0.98322635 0.98376154 0.98316306 0.98348652 0.98403691 0.98343998
 0.98377324 0.98332116 0.98389752 0.98293333 0.98353404 0.98357706
 0.98313672 0.98337687 0.9834047  0.98385422 0.98349379 0.98345116
 0.98391697 0.98350253]
9 4 [0.98322635 0.98376154 0.98316306 0.98348652 0.98403691 0.98343998
 0.98377324 0.98332116 0.98389752 0.98293333 0.98353404 0.98357706
 0.98313672 0.98337687 0.9834047  0.98385422 0.98349379 0.98345116
 0.98391697 0.98350253]
9 4 [0.98322635 0.98376154 0.98316306 0.98348652 0.98403691 0.98343998
 0.98377324 0

...

4 10 [0.98485441 0.98441437 0.98470067 0.98438661 0.98432145 0.98471368
 0.98466257 0.98471946 0.98447719 0.98437388 0.98495545 0.98445033
 0.98462919 0.98442062 0.98454951 0.98476983 0.98451623 0.98461714
 0.98464299 0.98458603]
4 10 [0.98485441 0.98441437 0.98470067 0.98438661 0.98432145 0.98471368
 0.98466257 0.98471946 0.98447719 0.98437388 0.98495545 0.98445033
 0.98462919 0.98442062 0.98454951 0.98476983 0.98471672 0.98461714
 0.98464299 0.98458603]
4 10 [0.98485441 0.98441437 0.98470067 0.98438661 0.98432145 0.98471368
 0.98466257 0.98471946 0.98447719 0.98437388 0.98495545 0.98445033
 0.98462919 0.98484411 0.98454951 0.98476983 0.98471672 0.98461714
 0.98464299 0.98458603]
4 10 [0.98485441 0.98441437 0.98470067 0.98438661 0.98432145 0.98471368
 0.98466257 0.98471946 0.98447719 0.98437388 0.98495545 0.98445033
 0.98462919 0.98484411 0.98454951 0.98476983 0.98471672 0.98461714
 0.98464299 0.98458603]
4 10 [0.98485441 0.98441437 0.98470067 0.98438661 0.98432145 0.98471368
 0.98466

9 13 [0.98485441 0.98481276 0.98470067 0.98447087 0.98454817 0.98471368
 0.98466257 0.98486674 0.98447719 0.98441944 0.98495545 0.98445033
 0.98462919 0.98501211 0.98462527 0.98476983 0.98471672 0.98461714
 0.98482118 0.98458603]
9 13 [0.98485441 0.98481276 0.98470067 0.98447087 0.98454817 0.98471368
 0.98466257 0.98486674 0.98447719 0.98441944 0.98495545 0.98445033
 0.98462919 0.98501211 0.98462527 0.98476983 0.98471672 0.98461714
 0.98482118 0.98458603]
9 13 [0.98485441 0.98481276 0.98470067 0.98447087 0.98454817 0.98471368
 0.98466257 0.98486674 0.98447719 0.98441944 0.98495545 0.98445033
 0.98462919 0.98501211 0.98462527 0.98476983 0.98471672 0.98461714
 0.98482118 0.98458603]
9 13 [0.98485441 0.98481276 0.98470067 0.98447087 0.98454817 0.98471368
 0.98466257 0.98486674 0.98447719 0.98441944 0.98495545 0.98445033
 0.98462919 0.98501211 0.98462527 0.98476983 0.98471672 0.98461714
 0.98482118 0.98458603]
9 13 [0.98485441 0.98481276 0.98470067 0.98447087 0.98454817 0.98471368
 0.98466

11 13 [0.98485441 0.98481276 0.98470106 0.98447087 0.98454817 0.9849107
 0.98466257 0.98486674 0.98458529 0.98485668 0.98495545 0.98445033
 0.9847764  0.98501211 0.98462527 0.98476983 0.9849692  0.98477626
 0.98482118 0.98479874]
11 13 [0.98485441 0.98481276 0.98470106 0.98447087 0.98454817 0.9849107
 0.98466257 0.98486674 0.98458529 0.98485668 0.98495545 0.98445033
 0.9847764  0.98501211 0.98462527 0.98476983 0.9849692  0.98477626
 0.98482118 0.98479874]
11 13 [0.98485441 0.98481276 0.98470106 0.98447087 0.98454817 0.9849107
 0.98466257 0.98486674 0.98458529 0.98485668 0.98495545 0.98445033
 0.9847764  0.98501211 0.98462527 0.98476983 0.9849692  0.98477626
 0.98482118 0.98479874]
11 13 [0.98485441 0.98481276 0.98470106 0.98447087 0.98454817 0.9849107
 0.98466257 0.98486674 0.98458529 0.98485668 0.98495545 0.98445033
 0.9848349  0.98501211 0.98462527 0.98476983 0.9849692  0.98477626
 0.98482118 0.98479874]
11 13 [0.98485441 0.98481276 0.98470106 0.98447087 0.98454817 0.9849107
 0.98466

11 9 [0.98485441 0.98483502 0.98470106 0.98461264 0.98456561 0.9849107
 0.98476726 0.98486674 0.98458529 0.98505346 0.98495545 0.98445033
 0.9848349  0.98501211 0.98487902 0.98487939 0.9849692  0.98477626
 0.98489458 0.98479874]
11 9 [0.98485441 0.98483502 0.98470106 0.98461264 0.98456561 0.9849107
 0.98476726 0.98486674 0.98458529 0.98505346 0.98495545 0.98445033
 0.9848349  0.98501211 0.98487902 0.98487939 0.9849692  0.98477626
 0.98489458 0.98479874]
11 9 [0.98485441 0.98483502 0.98470106 0.98461264 0.98456561 0.9849107
 0.98476726 0.98486674 0.98458529 0.98505346 0.98495545 0.98445033
 0.9848349  0.98501211 0.98487902 0.98487939 0.9849692  0.98477626
 0.98489458 0.98479874]
11 9 [0.98485441 0.98483502 0.98470106 0.98461264 0.98456561 0.9849107
 0.98476726 0.98486674 0.98458529 0.98505346 0.98495545 0.98445033
 0.9848349  0.98501211 0.98487902 0.98487939 0.9849692  0.98477626
 0.98489458 0.98479874]
11 9 [0.98485441 0.98483502 0.98470106 0.98461264 0.98456561 0.9849107
 0.98476726 0

4 9 [0.98485441 0.98483502 0.98504808 0.98461264 0.98456561 0.9849107
 0.98495949 0.98486674 0.98497043 0.98505346 0.98495545 0.98500457
 0.9848349  0.98501211 0.98487902 0.98487939 0.9849692  0.98477626
 0.98489458 0.98479874]
4 9 [0.98485441 0.98483502 0.98504808 0.98461264 0.98456561 0.9849107
 0.98502776 0.98486674 0.98497043 0.98505346 0.98495545 0.98500457
 0.9848349  0.98501211 0.98487902 0.98487939 0.9849692  0.98477626
 0.98489458 0.98479874]
3 9 [0.98489034 0.98483502 0.98504808 0.98461264 0.98487777 0.9849107
 0.98502776 0.98486674 0.98497043 0.98505346 0.98495545 0.98500457
 0.9848349  0.98501211 0.98487902 0.98487939 0.9849692  0.98477626
 0.98489458 0.98479874]
3 9 [0.98489034 0.98483502 0.98504808 0.98461264 0.98487777 0.9849107
 0.98502776 0.98486674 0.98497043 0.98505346 0.98495545 0.98500457
 0.9848349  0.98501211 0.98487902 0.98487939 0.9849692  0.98477626
 0.98489458 0.98479874]
3 9 [0.98489034 0.98483502 0.98504808 0.98461264 0.98487777 0.9849107
 0.98502776 0.9848

In [44]:
drawAll_alt(data_pre,data_test)
result_alt(data_pre,data_test,'jaya_opt_alt')

NameError: name 'drawAll_alt' is not defined

In [32]:
print(e)

[[ 0.         -2.39677804  1.53224693  1.51741685 -0.04307962]
 [-1.74363629  0.         -1.27885496  0.99677682 -1.31961101]
 [-1.70582904 -0.67366926  0.         -0.1993561  -2.53269137]
 [-0.2676375  -3.13287974 -1.06136114  0.         -0.6436346 ]
 [-1.49306341  3.29809459  1.89944673 -1.39211663  0.        ]]
